# Assignment 6 - Text Classification Pipeline

In this task you will apply the concepts learned in the last sessions:

- Feature Extraction for Text data
- scikit-learn pipelines
- ML Model Evaluation
- Cross-Validation

As all these things are too complex to implement in an exercise, we will be using existing libraries for this, in particular ``scikit-learn``.

We also will make use of a library for storing tabular data, ``pandas``, our data will be stored as pandas dataframe. You don't need to understand much about this format, except for how to retrieve a column from it. Just like for dictionaries you can type ``df[colname]`` to get the column values.

The first cell downloads some text data, it's a number of speeches in the German Parliament, the *Bundestag*.

Your task will be to predict party affiliation from the speech text.

You will do that by building a sklearn pipeline and use grid search to optimize the n-gram range of thee text featurizer.

You can check whether your code worked if it produces the same classification reports as the solution.

In [ ]:
import os, gzip
import pandas as pd
import numpy as np
import urllib.request

np.random.seed(0)

import warnings
warnings.filterwarnings('ignore')

DATADIR = "data"

if not os.path.exists(DATADIR):
    os.mkdir(DATADIR)

file_name = os.path.join(DATADIR, 'bundestags_parlamentsprotokolle.csv.gzip')
if not os.path.exists(file_name):
    url_data = 'https://www.dropbox.com/s/1nlbfehnrwwa2zj/bundestags_parlamentsprotokolle.csv.gzip?dl=1'
    urllib.request.urlretrieve(url_data, file_name)

df = pd.read_csv(gzip.open(file_name), index_col=0).sample(n=10000)

We can inspect the first 4 rows of the pandas dataframe like this

In [ ]:
df[:4]

,sitzung,wahlperiode,sprecher,text,partei
17908,201,17,Ulrich Lange,Es ist vielmehr – das erlaube ich mir hier sch...,cducsu
42100,226,18,Doris Wagner,"Ein weiteres Finanzierungsinstrument, um diese...",gruene
18777,209,17,Heidrun Bluhm,In den letzten zwei Jahrzehnten sind immer meh...,linke
14589,168,17,Dr. Egon Jüttner,– Ja. – Von rund 80 000 bis 100 000 Herero im ...,cducsu


Your solution will have to implement the following parts:

- Split the data into train (80%) and test (20%) set. You can use the sklearn function ``train_test_split`` for this.

- Build a pipeline with a [``TfidfVectorizer``](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and a [``NearestCentroid``](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html) classifier

- Train the pipeline inside a [``GridSearchCV``](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) object on the training data. Try to find the best ``n_gram_range`` for the Vectorizer.

- Evaluate F1, precision, recall on the test data. You can use the sklearn function ``classification_report`` for that.

In [ ]:
# uncomment these lines to install the required dependencies.
# !pip install numpy
# !pip install matplotlib
# !pip install scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import NearestCentroid
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
'''MY CODE:'''
# global
'''
- X werden die 'text' aus df gegeben
- y werden die 'partei' aus df gegeben
- die Trainings und Test data(Text) und Labels(Partei) werden definiert und
  mit train_test_split werden die Test Dateien auf 20% kalibriert

- die Pipeline hat 2 Stufen:
  1. tfidf: konvertiert die Textdateien in Vektoren (one_hot_encoding)
            die Häufigkeit von Wörtern oder Wortkombinationen werden dargestellt
  2. classifier: weist die Dokumente anhand den Durchschnittsvektoren zu der
                  jeweiligen Klasse
'''
X = df['text']
y = df['partei']
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.2)

clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', NearestCentroid())
])

# train_model
'''
- param_grid definiert ein Parameterraster in welchem tfidf prüfen soll, also in
  welchem Raster oder Wortgruppen es geprüft werden soll

- grid_search erstellt ein objekt:
  die Funktionen in der Pipeline werden genutzt
  das Modell wir also 3 mal trainiert(mit jedem grid)
  cv=5 beudeutet dass es 5 mal auf einen Teil trainiert und auf einen anderen
  Trainingsteil getestet wird

- grid_search.fit hier wird das search Objekt dann wirklich genutzt

- best_params, best_model hier wird gespeichert welcher grid am besten ist und
                          und welches Modell nach den (cv=5) durchgängen

- return: das Modell mit den besten Ergebnissen + die test Dateien mit den getestet wurde
  es wird also erst mit den Trainingsdaten dem Modell beigebracht wie es zu arbeiten hat
  und dann mit den Testdaten bewertet wie gut es funktioniert
'''
def train_model(X, y):
    param_grid = {
        'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)]
    }

    grid_search = GridSearchCV(clf, param_grid, cv=5)
    grid_search.fit(train_data, train_labels)
    best_params, best_model = grid_search.best_params_, grid_search.best_estimator_

    return best_model

'''
alles was aus der Funktion returned wurde wird hier abgespeichert
model: das beste Modell
'''
model = train_model(train_data, train_labels)

# hier wird das trainierte Modell genutzt um die labels der test_data vorherzusagen
predictions = model.predict(test_data)

'''
hier wird ein Bericht erstellt der die Daten zusammenfasst
- precision: der Anteil der korrekt vorhergesagt wurde
- recall: der Anteil der als positiv vorhergesagt und auch wirklich positiv war
- f1: der Mmittelwert von precision und recall
- support: normalisiert die werte damit es prozentual gleich bleibt auch bei mehr Samples
'''
class_report = classification_report(test_labels, predictions)
print(class_report)
'''END'''

              precision    recall  f1-score   support

      cducsu       0.58      0.73      0.65       742
         fdp       0.36      0.07      0.12       145
      gruene       0.44      0.31      0.36       323
       linke       0.53      0.43      0.48       261
         spd       0.39      0.44      0.41       529

    accuracy                           0.50      2000
   macro avg       0.46      0.40      0.40      2000
weighted avg       0.48      0.50      0.48      2000



### Comparision with Training dataset

In [ ]:
ncc_predictions = model.predict(train_data)
print(classification_report(ncc_predictions, train_labels))

              precision    recall  f1-score   support

      cducsu       0.97      0.97      0.97      2870
         fdp       1.00      1.00      1.00       637
      gruene       0.97      0.97      0.97      1195
       linke       0.98      0.94      0.96      1132
         spd       0.97      0.98      0.98      2166

    accuracy                           0.97      8000
   macro avg       0.98      0.97      0.98      8000
weighted avg       0.97      0.97      0.97      8000



### Comparision with Testing dataset

In [ ]:
ncc_predictions_test = model.predict(test_data)
print(classification_report(ncc_predictions_test, test_labels))

              precision    recall  f1-score   support

      cducsu       0.73      0.58      0.65       939
         fdp       0.07      0.36      0.12        28
      gruene       0.31      0.44      0.36       224
       linke       0.43      0.53      0.48       210
         spd       0.44      0.39      0.41       599

    accuracy                           0.50      2000
   macro avg       0.40      0.46      0.40      2000
weighted avg       0.56      0.50      0.52      2000

